In [2]:
import os
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

def generate_random_key(key_length):
    return os.urandom(key_length // 8)

def aes_encrypt(key, plaintext, mode, iv=None):
    if mode == 'CBC':
        if iv is None:
            iv = os.urandom(16)
        encryptor = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend()).encryptor()
    elif mode == 'ECB':
        encryptor = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).encryptor()
    else:
        raise ValueError("Invalid mode specified")
    
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    
    return iv, ciphertext

def aes_decrypt(key, ciphertext, mode, iv):
    if mode == 'CBC':
        decryptor = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend()).decryptor()
    elif mode == 'ECB':
        decryptor = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).decryptor()
    else:
        raise ValueError("Invalid mode specified")
    
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()
    
    return plaintext

# Example usage:
if __name__ == "__main__":
    key_length = 256  # AES key length in bits
    key = generate_random_key(key_length)
    plaintext = b"Hello, AES encryption!"

    iv, ciphertext = aes_encrypt(key, plaintext, mode='CBC')
    print("IV:", iv.hex())
    print("Ciphertext:", ciphertext.hex())

    decrypted_plaintext = aes_decrypt(key, ciphertext, mode='CBC', iv=iv)
    print("Decrypted plaintext:", decrypted_plaintext.decode('utf-8'))


IV: 72fe006810567e6da63c68ac9076c982
Ciphertext: c035ad11359ae73028ab36261ced50a251a1197980839cdf42f3f8124c1dedc5
Decrypted plaintext: Hello, AES encryption!
